In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,830 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [937 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.co

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
from google.colab import files
uploaded = files.upload()

Saving homework_source_file.zip to homework_source_file.zip


In [5]:
!unzip homework_source_file.zip

Archive:  homework_source_file.zip
  inflating: suppliers.csv           
  inflating: order_details.csv       
  inflating: orders.csv              
  inflating: products.csv            


In [6]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df_products = spark.read. \
  format("csv"). \
  option("inferSchema","true"). \
  option("header","true"). \
  load("products.csv")

In [9]:
df_products.show(2)
df_products.printSchema()
df_products.dtypes

+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|product_id|product_name|supplier_id|category_id| quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|         1|        Chai|          8|          1|10 boxes x 30 bags|      18.0|            39|             0|           10|           1|
|         2|       Chang|          1|          1|24 - 12 oz bottles|      19.0|            17|            40|           25|           1|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
only showing top 2 rows

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- supplier_id: integer (nullable = true)
 |-- category_id:

[('product_id', 'int'),
 ('product_name', 'string'),
 ('supplier_id', 'int'),
 ('category_id', 'int'),
 ('quantity_per_unit', 'string'),
 ('unit_price', 'double'),
 ('units_in_stock', 'int'),
 ('units_on_order', 'int'),
 ('reorder_level', 'int'),
 ('discontinued', 'int')]

In [10]:
df_products = df_products \
  .withColumn("product_id" ,
              df_products["product_id"]
              .cast(StringType()))   \
  .withColumn("supplier_id",
              df_products["supplier_id"]
              .cast(StringType()))    \
  .withColumn("category_id"  ,
              df_products["category_id"]
              .cast(StringType())) \

In [11]:
df_products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- supplier_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- quantity_per_unit: string (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- units_in_stock: integer (nullable = true)
 |-- units_on_order: integer (nullable = true)
 |-- reorder_level: integer (nullable = true)
 |-- discontinued: integer (nullable = true)



In [12]:
df_orders = spark.read. \
  format("csv"). \
  option("inferSchema","true"). \
  option("header","true"). \
  load("orders.csv")

In [13]:
df_orders.show(2)
df_orders.printSchema()
df_orders.dtypes

+--------+-----------+-----------+-------------------+-------------------+-------------------+--------+-------+--------------------+------------------+---------+-----------+----------------+------------+
|order_id|customer_id|employee_id|         order_date|      required_date|       shipped_date|ship_via|freight|           ship_name|      ship_address|ship_city|ship_region|ship_postal_code|ship_country|
+--------+-----------+-----------+-------------------+-------------------+-------------------+--------+-------+--------------------+------------------+---------+-----------+----------------+------------+
|   10248|      VINET|          5|1996-07-04 00:00:00|1996-08-01 00:00:00|1996-07-16 00:00:00|       3|  32.38|Vins et alcools C...|59 rue de l'Abbaye|    Reims|       null|           51100|      France|
|   10249|      TOMSP|          6|1996-07-05 00:00:00|1996-08-16 00:00:00|1996-07-10 00:00:00|       1|  11.61|  Toms Spezialitäten|     Luisenstr. 48|  Münster|       null|           

[('order_id', 'int'),
 ('customer_id', 'string'),
 ('employee_id', 'int'),
 ('order_date', 'timestamp'),
 ('required_date', 'timestamp'),
 ('shipped_date', 'timestamp'),
 ('ship_via', 'int'),
 ('freight', 'double'),
 ('ship_name', 'string'),
 ('ship_address', 'string'),
 ('ship_city', 'string'),
 ('ship_region', 'string'),
 ('ship_postal_code', 'string'),
 ('ship_country', 'string')]

In [14]:
df_orders = df_orders \
  .withColumn("order_id" ,
              df_orders["order_id"]
              .cast(StringType()))   \
  .withColumn("employee_id"  ,
              df_orders["employee_id"]
              .cast(StringType())) \

In [15]:
df_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- employee_id: string (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- required_date: timestamp (nullable = true)
 |-- shipped_date: timestamp (nullable = true)
 |-- ship_via: integer (nullable = true)
 |-- freight: double (nullable = true)
 |-- ship_name: string (nullable = true)
 |-- ship_address: string (nullable = true)
 |-- ship_city: string (nullable = true)
 |-- ship_region: string (nullable = true)
 |-- ship_postal_code: string (nullable = true)
 |-- ship_country: string (nullable = true)



In [21]:
df_order_details = spark.read. \
  format("csv"). \
  option("inferSchema","true"). \
  option("header","true"). \
  load("order_details.csv")

In [22]:
df_order_details.show(2)
df_order_details.printSchema()
df_order_details.dtypes

+--------+----------+----------+--------+--------+
|order_id|product_id|unit_price|quantity|discount|
+--------+----------+----------+--------+--------+
|   10248|        11|      14.0|      12|     0.0|
|   10248|        42|       9.8|      10|     0.0|
+--------+----------+----------+--------+--------+
only showing top 2 rows

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- discount: double (nullable = true)



[('order_id', 'int'),
 ('product_id', 'int'),
 ('unit_price', 'double'),
 ('quantity', 'int'),
 ('discount', 'double')]

In [23]:
df_order_details = df_order_details \
  .withColumn("order_id" ,
              df_order_details["order_id"]
              .cast(StringType()))   \
  .withColumn("product_id"  ,
              df_order_details["product_id"]
              .cast(StringType())) \

In [24]:
df_order_details.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- discount: double (nullable = true)



In [25]:
df_suppliers = spark.read. \
  format("csv"). \
  option("inferSchema","true"). \
  option("header","true"). \
  load("suppliers.csv")

In [26]:
df_suppliers.show(2)
df_suppliers.printSchema()
df_suppliers.dtypes

+-----------+--------------------+----------------+-------------------+--------------+-----------+------+-----------+-------+--------------+----+-----------+
|supplier_id|        company_name|    contact_name|      contact_title|       address|       city|region|postal_code|country|         phone| fax|   homepage|
+-----------+--------------------+----------------+-------------------+--------------+-----------+------+-----------+-------+--------------+----+-----------+
|          1|      Exotic Liquids|Charlotte Cooper| Purchasing Manager|49 Gilbert St.|     London|  null|    EC1 4SD|     UK|(171) 555-2222|null|       null|
|          2|New Orleans Cajun...|   Shelley Burke|Order Administrator|P.O. Box 78934|New Orleans|    LA|      70117|    USA|(100) 555-4822|null|#CAJUN.HTM#|
+-----------+--------------------+----------------+-------------------+--------------+-----------+------+-----------+-------+--------------+----+-----------+
only showing top 2 rows

root
 |-- supplier_id: inte

[('supplier_id', 'int'),
 ('company_name', 'string'),
 ('contact_name', 'string'),
 ('contact_title', 'string'),
 ('address', 'string'),
 ('city', 'string'),
 ('region', 'string'),
 ('postal_code', 'string'),
 ('country', 'string'),
 ('phone', 'string'),
 ('fax', 'string'),
 ('homepage', 'string')]

In [93]:
df_suppliers = df_suppliers \
  .withColumn("supplier_id" ,
              df_suppliers["supplier_id"]
              .cast(StringType()))   \

In [50]:
df_suppliers.dtypes

[('supplier_id', 'string'),
 ('company_name', 'string'),
 ('contact_name', 'string'),
 ('contact_title', 'string'),
 ('address', 'string'),
 ('city', 'string'),
 ('region', 'string'),
 ('postal_code', 'string'),
 ('country', 'string'),
 ('phone', 'string'),
 ('fax', 'string'),
 ('homepage', 'string')]

In [68]:
df_denormalized = df_orders.join(df_order_details,"order_id").join(df_products,"product_id").join(df_suppliers,"supplier_id")

In [69]:
df_denormalized.show(10)

+-----------+----------+--------+-----------+-----------+-------------------+-------------------+-------------------+--------+-------+--------------------+------------------+--------------+-----------+----------------+------------+----------+--------+--------+--------------------+-----------+-----------------+----------+--------------+--------------+-------------+------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-----------+---------+--------------+--------------+--------------------+
|supplier_id|product_id|order_id|customer_id|employee_id|         order_date|      required_date|       shipped_date|ship_via|freight|           ship_name|      ship_address|     ship_city|ship_region|ship_postal_code|ship_country|unit_price|quantity|discount|        product_name|category_id|quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|        company_name|        contact_name|       contact_title|

In [84]:
df_denormalized.show(2)
df_denormalized.printSchema()
df_denormalized.dtypes

+-----------+----------+--------+-----------+-----------+-------------------+-------------------+-------------------+--------+-------+--------------------+------------------+---------+-----------+----------------+------------+----------+--------+--------+--------------------+-----------+-----------------+----------+--------------+--------------+-------------+------------+--------------------+------------+--------------------+--------------------+---------+------+-----------+---------+------------+------------+--------------+
|supplier_id|product_id|order_id|customer_id|employee_id|         order_date|      required_date|       shipped_date|ship_via|freight|           ship_name|      ship_address|ship_city|ship_region|ship_postal_code|ship_country|unit_price|quantity|discount|        product_name|category_id|quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|        company_name|contact_name|       contact_title|             address|     city|region|po

[('supplier_id', 'string'),
 ('product_id', 'string'),
 ('order_id', 'string'),
 ('customer_id', 'string'),
 ('employee_id', 'string'),
 ('order_date', 'timestamp'),
 ('required_date', 'timestamp'),
 ('shipped_date', 'timestamp'),
 ('ship_via', 'int'),
 ('freight', 'double'),
 ('ship_name', 'string'),
 ('ship_address', 'string'),
 ('ship_city', 'string'),
 ('ship_region', 'string'),
 ('ship_postal_code', 'string'),
 ('ship_country', 'string'),
 ('unit_price', 'double'),
 ('quantity', 'int'),
 ('discount', 'double'),
 ('product_name', 'string'),
 ('category_id', 'string'),
 ('quantity_per_unit', 'string'),
 ('unit_price', 'double'),
 ('units_in_stock', 'int'),
 ('units_on_order', 'int'),
 ('reorder_level', 'int'),
 ('discontinued', 'int'),
 ('company_name', 'string'),
 ('contact_name', 'string'),
 ('contact_title', 'string'),
 ('address', 'string'),
 ('city', 'string'),
 ('region', 'string'),
 ('postal_code', 'string'),
 ('country', 'string'),
 ('phone', 'string'),
 ('fax', 'string'),
 

In [148]:
net_revenue_formula = (df_order_details["quantity"]*(df_order_details["unit_price"]) - (1-df_order_details["discount"]))
df_net_revnue_unsorted = df_denormalized.groupBy("supplier_id",("contact_name"),year("order_date").alias("Year")).agg(round(sum(net_revenue_formula),2).alias("Net_revenue"))

In [149]:
df_net_revnue_unsorted.orderBy("Supplier_id","contact_name","Year").show(10)

+-----------+----------------+----+-----------+
|supplier_id|    contact_name|Year|Net_revenue|
+-----------+----------------+----+-----------+
|          1|Charlotte Cooper|1996|    3667.25|
|          1|Charlotte Cooper|1997|     9336.4|
|          1|Charlotte Cooper|1998|    8584.25|
|         10|     Carlos Diaz|1996|     558.05|
|         10|     Carlos Diaz|1997|     1739.0|
|         10|     Carlos Diaz|1998|    2437.45|
|         11|   Petra Winkler|1996|     6174.1|
|         11|   Petra Winkler|1997|   23834.17|
|         11|   Petra Winkler|1998|   10754.18|
|         12|     Martin Bein|1996|    21757.2|
+-----------+----------------+----+-----------+
only showing top 10 rows



In [150]:
data = df_net_revnue_unsorted.orderBy("Supplier_id","contact_name","Year")

In [151]:
df = data.selectExpr("supplier_id as Supplier_id","contact_name as Supplier_name","Year","Net_revenue")

In [152]:
df.show(20)

+-----------+----------------+----+-----------+
|Supplier_id|   Supplier_name|Year|Net_revenue|
+-----------+----------------+----+-----------+
|          1|Charlotte Cooper|1996|    3667.25|
|          1|Charlotte Cooper|1997|     9336.4|
|          1|Charlotte Cooper|1998|    8584.25|
|         10|     Carlos Diaz|1996|     558.05|
|         10|     Carlos Diaz|1997|     1739.0|
|         10|     Carlos Diaz|1998|    2437.45|
|         11|   Petra Winkler|1996|     6174.1|
|         11|   Petra Winkler|1997|   23834.17|
|         11|   Petra Winkler|1998|   10754.18|
|         12|     Martin Bein|1996|    21757.2|
|         12|     Martin Bein|1997|   69242.53|
|         12|     Martin Bein|1998|    64776.7|
|         13|   Sven Petersen|1996|     3140.0|
|         13|   Sven Petersen|1997|     6540.1|
|         13|   Sven Petersen|1998|    5066.04|
|         14|      Elio Rossi|1996|    13006.9|
|         14|      Elio Rossi|1997|    22870.9|
|         14|      Elio Rossi|1998|    1